In [142]:
pip install catboost



  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=e1d303b107afe7feaecb067c0156cb414dc9fa76a6546c3f57623ed42f420cee
  Stored in directory: c:\users\ualkhanova_a\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [171]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from tqdm import tqdm
from importlib import reload
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression
from catboost import CatBoostRegressor

import os
print(os.listdir('./data'))
PATH_to_file = './data/'

['all_auto_ru_09_09_2020.csv', 'all_auto_ru_24_08_2020.csv', 'all_car_brands_and_their_models_from_auto_ru.csv', 'auto_ru.csv', 'df3.csv', 'df3_cols.csv', 'sample_submission.csv', 'test.csv']


In [172]:
import utils_module09092020 as utils

In [173]:
VAL_SIZE = 0.33 # 33%
RANDOM_SEED = 42

In [228]:
# это блок закомментирован и требуется для закачивания готового датасета

df3 = pd.read_csv(PATH_to_file+'df3.csv')
df_test = pd.read_csv(PATH_to_file+'test.csv')
df_submit = pd.read_csv(PATH_to_file+'sample_submission.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', df3.shape)
display(df3.head(2))
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head(2))
print('Размерность датасета c примером сабмишена: ', df_submit.shape)
display(df_submit.head(2))

Размерность тренировочного датасета:  (52085, 30)


,bodyType,brand,color,fuelType,vehicleTransmission,Привод,ПТС,model2,mileage,modelDate,productionDate,enginePower,engineDisplacement2,engineDisplacement2_log,numberOfDoors,Владельцы,count_words_d,mean_c_w,sum_c_w,dif_b_prod_model,Train,id,c_p_des1,m_s_w_des2,r_l_s_des3,abs_des4,c_c_des5,a_w_des6,r_v_c_des7,price_log
0,седан,AUDI,чёрный,бензин,MECHANICAL,полный,Оригинал,2.8,350000,30,29,174,2800,7,4,3,32,0.000331,30,-1,1,NaN,0,0,0,0,0,0,0,12.206073
1,седан,AUDI,красный,бензин,MECHANICAL,передний,Оригинал,1.8,173424,38,34,90,1800,7,4,3,21,0.000199,18,-4,1,NaN,0,0,0,0,0,0,0,11.002100


Размерность тестового датасета:  (3837, 23)


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
0,седан,BMW,чёрный,дизель,2016.0,520d 2.0d AT (190 л.с.),4.0,2017.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,190 N12,В РОЛЬФ Ясенево представлено более 500 автомоб...,158836.0,"['[{""name"":""Безопасность"",""values"":[""Антипробу...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,0
1,седан,BMW,белый,дизель,2018.0,318d 2.0d AT (150 л.с.),4.0,2019.0,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,150 N12,Автомобиль из демонстрационного парка по спец ...,10.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,1


Размерность датасета c примером сабмишена:  (3837, 2)


,id,price
0,0,5299000.0
1,1,1580000.0


In [229]:
# создаем думми переменные из категориальных переменных
df3 = pd.get_dummies(df3, columns = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'Привод', 'Владельцы', 'ПТС', 'model2', 'engineDisplacement2'])

In [230]:
train = df3.query('Train==1').drop(['Train'], axis = 1)
test = df3.query('Train==0').drop(['Train'], axis = 1)

In [231]:
X_train = train.drop(['price_log', 'id'], axis=1).reset_index(drop=True)
y_train = train['price_log'].values
X_test = test.drop(['price_log', 'id'], axis=1).reset_index(drop=True)
y_test = test['price_log'].values

In [232]:
# Проверим RandomForestRegressor
rf  = RandomForestRegressor(random_state=RANDOM_SEED) 
rf.fit(X, y)
y_train_predict = rf.predict(X)
y_predict = rf.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")


 Train MAPE: 0.006
 Test MAPE: 0.552


In [221]:
#попробуем поискать лучшие значения для случайного леса
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=RANDOM_SEED, n_jobs = -1)
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 41.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 177.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 353.6min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [222]:
rf_random.best_params_

{'n_estimators': 1200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 110,
 'bootstrap': True}

In [233]:
# Проверим RandomForestRegressor с лучшими параметрами
rf  = RandomForestRegressor(random_state=RANDOM_SEED, n_estimators = 1200, min_samples_split = 2, min_samples_leaf = 1, 
                            max_features = 'sqrt', max_depth = 110, bootstrap = True) 
rf.fit(X_train, y_train)
y_train_predict = rf.predict(X_train)
y_predict = rf.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 0.006
 Test MAPE: 0.464


In [235]:
#что то результат не очень 46,4% получим Submission  
Sub = np.exp(y_predict)
submit = pd.DataFrame(test.id)
submit['default']=Sub
submit.to_csv('submission_RF.csv', index=False)

In [236]:
# Проверим DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RANDOM_SEED)
dtr.fit(X_train, y_train)
y_train_predict = dtr.predict(X_train)
y_predict = dtr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")



 Train MAPE: 0.000
 Test MAPE: 0.590


In [238]:
# Проверим LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_predict = lr.predict(X_train)
y_predict = lr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")


 Train MAPE: 0.072
 Test MAPE: 545100.493


In [239]:
# проверим BaggingClassifier - делит выборку на бутстрап выборки а потом из нескольких моделей создает усредненный результат
BagClas_lr = BaggingRegressor(lr, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
BagClas_lr.fit(X_train, y_train)
y_train_predict = BagClas_lr.predict(X_train)
y_predict = BagClas_lr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 870.438
 Test MAPE: 38963127.032


In [240]:
# проверим градиентный бустинг
gbr = GradientBoostingRegressor(n_estimators=300)
gbr.fit(X_train, y_train)
y_train_predict = gbr.predict(X_train)
y_predict = gbr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 0.046
 Test MAPE: 0.553


In [ ]:
# Проверим GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state = RANDOM_SEED)
gbr.fit(X_train, y_train)

param_grid = {'learning_rate':[0.00001, 0.0001, 0.001, 0.01, 0.1, 1],
             'n_estimators':[100, 250, 500, 750, 1000, 1250, 1500, 1750]}

rf_random = GridSearchCV(estimator=gbr, param_grid=param_grid, cv=5, n_jobs=-1)
rf_random.fit(X_train, y_train)
print(rf_random.best_params_)
print(rf_random.best_score_)

In [115]:
rf = GradientBoostingRegressor(random_state = RANDOM_SEED, learning_rate = 0.1, n_estimators = 750)
rf.fit(X_train, y_train)
y_train_predict = rf.predict(X_train)
y_predict = rf.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 0.012
 Test MAPE: 0.048


In [116]:
# проверить ExtraTreeRegressor
etr = ExtraTreeRegressor(random_state = RANDOM_SEED)
etr.fit(X_train, y_train)
y_train_predict = etr.predict(X_train)
y_predict = etr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 0.000
 Test MAPE: 0.081


In [140]:
#проверим для него лучшие параметры
param_grid = {
     #'n_estimators': [100],
     'max_features': ['auto', 'sqrt', 'log2']
}

etr = ExtraTreeRegressor(random_state = RANDOM_SEED)
etr_random = GridSearchCV(estimator=etr, param_grid=param_grid, cv=5, n_jobs=-1)
etr_random.fit(X_train, y_train)
print(etr_random.best_params_)


{'max_features': 'auto'}


In [141]:
etr = ExtraTreeRegressor(random_state = RANDOM_SEED, max_features = 'auto')
etr.fit(X_train, y_train)
y_train_predict = etr.predict(X_train)
y_predict = etr.predict(X_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")

 Train MAPE: 0.000
 Test MAPE: 0.081


In [151]:
# проверим стэкинг с GradientBoostingRegressor c наилучшими парамерами, RandomForestRegressor 

def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features using the classifier.
    
    :arg clf: scikit-learn classifier
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

def generate_metafeatures(classifiers, X_train, X_test, y_train, cv):
    """
    Generates metafeatures using a list of classifiers.
    
    :arg classifiers: list of scikit-learn classifiers
    :args X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

cv = KFold(n_splits=10, shuffle=True)

stacked_features_train, stacked_features_test = generate_metafeatures([
    RandomForestRegressor(random_state=RANDOM_SEED, n_estimators = 1200, min_samples_split = 2, min_samples_leaf = 1, 
                            max_features = 'sqrt', max_depth = 110, bootstrap = True),
    GradientBoostingRegressor(random_state = RANDOM_SEED, learning_rate = 0.1, n_estimators = 750)
], X_train, X_test, y_train, cv)

clf = LinearRegression()

clf.fit(stacked_features_train, y_train)
y_train_predict = clf.predict(stacked_features_train)
y_predict = rf.predict(stacked_features_test)
print(f" Train MAPE: {mean_squared_error(y_train, y_train_predict):0.3f}")
print(f" Test MAPE: {mean_squared_error(y_test, y_predict):0.3f}")


  0%|          | 0/2 [00:00<?, ?it/s]

KeyError: "None of [Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,\n            ...\n            4295, 4296, 4297, 4298, 4299, 4300, 4301, 4302, 4303, 4304],\n           dtype='int64', length=3874)] are in the [columns]"